# Install the virtual environement

# Important librairies

In [1]:
#pip install -U spacy
#python -m spacy download en_core_web_trf

#Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import Image
from lxml import objectify
import math
import time
#import neuralcoref
import dateparser
import re
import csv
import coreferee, spacy
import fnmatch
#from pytrends.request import TrendReq
from wordcloud import WordCloud
#from ast import literal_eval
from collections import Counter
from spacy import displacy
from spacy.tokens import Span
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import utils
from spacy.tokens import Doc
from itertools import groupby
from tqdm import tqdm
import tqdm as notebook_tqdm

import spacy

#Import data from wikidata.org
#from wikidata2df import wikidata2df

#Convert country name to ISO-3
import country_converter as coco

#World map plot librairies
import plotly.offline as po
import plotly.graph_objs as pg
import plotly.express as px
import plotly.graph_objs as go

# Analysis of the lexical fields
from empath import Empath 

#Warnings
import warnings

data_folder = './data/'

c:\Users\SP80_record\.conda\envs\ADA_3.10\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Import Data

## Data from movies and characters

In [2]:
df_movie_usa = pd.read_csv('df_movie_usa.csv')
df_character_usa = pd.read_csv('df_character_usa.csv')

decades = ['1930-1940','1940-1950','1950-1960','1960-1970','1970-1980','1980-1990','1990-2000','2000-2010']
decades_code = range(len(decades))
df_movie_usa['Decade'] = 0
for ctr,decade in enumerate(decades):
    index1 = df_movie_usa[ (df_movie_usa['Movie_date']>decade[0:3]) & (df_movie_usa['Movie_date']<decade[5:8])].index
    df_movie_usa.loc[index1,'Decade'] = decades_code[ctr]
df_movie_usa

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,TS score_x,...,positive_emotion,negative_emotion,Number_of_words,he_count,she_count,Principal_summary_character,Secondary_summary_character,Principal_character_sex,Principal_character_actor,Decade
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",22%,...,0.0,7.0,396.0,1.0,0.0,Ballard,Williams,F,Natasha Henstridge,7
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...",['United States of America'],"['Short Film', 'Silent film', 'Indie', 'Black-...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Musical', 'Comedy', 'Black-and-white']",80%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Drama', 'Black-and-white']",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34403,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","['France', 'United States of America']","['Parody', 'Americana', 'Comedy']",85%,...,2.0,1.0,252.0,1.0,1.0,Carla,Walt,F,Julia Stiles,7
34404,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},['United States of America'],"['Thriller', 'Erotic thriller', 'Psychological...",38%,...,1.0,9.0,611.0,6.0,8.0,Greenhill,Haines,M,Don Johnson,6
34405,24209227,/m/07k5mlk,"The Time, the Place and the Girl",1946,NaN,105.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Comedy film', 'Romance Film', 'Musical']",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
34406,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],['Drama'],NaN,...,1.0,0.0,118.0,0.0,0.0,NaN,NaN,NaN,NaN,0


## Data having summaries

In [3]:
#Dataframe with existing movie summary
wiki_id_list = []
for ctr,wiki_id in enumerate((df_movie_usa['Wikipedia_movie_ID'])) : 
    path = data_folder+'/corenlp_plot_summaries/'+str(wiki_id)+'.xml'
    if os.path.exists(path):
        wiki_id_list.append(wiki_id)
wiki_id_series = pd.Series(wiki_id_list, name='Wikipedia_movie_ID',dtype=object)

#Data frame (movie metadata) with only existing summaries 
df_movie_usa_summary=df_movie_usa.merge(wiki_id_series, on='Wikipedia_movie_ID')


#Data frame (character metadata) with only existing summaries
df_character_usa_summary=df_character_usa.merge(wiki_id_series, on='Wikipedia_movie_ID')

In [4]:
df_character_usa_summary

,index,Wikipedia_movie_ID,Freebase_movie_ID,Movie_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character_actor_ID,Freebase_character_ID,Freebase_actor_ID,ethnicLabel
0,0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,NaN
1,1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,NaN
2,2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans
3,3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,NaN
4,4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185413,451437,30553937,/m/0g9ts5h,1998-03-07,Steve,1959-04-02,M,NaN,NaN,Steve Monarque,38.0,/m/0jxtp_5,/m/0jxtp_8,/m/0bl1p7,NaN
185414,451438,30553937,/m/0g9ts5h,1998-03-07,Mark,NaN,M,NaN,NaN,Kevin Richardson,NaN,/m/0jxtpzm,/m/0jxtpzq,/m/0jxtpzy,NaN
185415,451439,30553937,/m/0g9ts5h,1998-03-07,Barbara,NaN,F,NaN,NaN,Leda Mulholland,NaN,/m/0jxtsvw,/m/0jxtsvz,/m/0gbyr6p,NaN
185416,451440,30553937,/m/0g9ts5h,1998-03-07,Restaurant Manager,NaN,M,NaN,NaN,Randy Harris,NaN,/m/0jxtsx5,/m/0jxtsx8,/m/0jxtsxh,NaN


In [5]:
df_movie_usa_summary

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,TS score_x,...,positive_emotion,negative_emotion,Number_of_words,he_count,she_count,Principal_summary_character,Secondary_summary_character,Principal_character_sex,Principal_character_actor,Decade
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",22%,...,0.0,7.0,396.0,1.0,0.0,Ballard,Williams,F,Natasha Henstridge,7
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Romantic comedy', 'Ensemble Film', 'Comedy-d...",NaN,...,2.0,2.0,257.0,2.0,5.0,Adam,Kate,NaN,NaN,6
2,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Science Fiction', 'Adventure', 'Animation', ...",NaN,...,0.0,3.0,707.0,4.0,2.0,Lois,Clark,NaN,NaN,0
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"[""Children's/Family"", 'Musical', 'Fantasy', 'C...",NaN,...,8.0,5.0,1678.0,12.0,7.0,Mary,Banks,F,Julie Andrews,3
4,21926710,/m/05p45cv,White on Rice,2009,NaN,82.0,{},['United States of America'],"['Romantic comedy', 'Romance Film', 'Comedy', ...",45%,...,2.0,0.0,125.0,3.0,0.0,Jimmy,Tak,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20784,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}",['United States of America'],"['Alien Film', 'Science Fiction', 'Family Film...",8%,...,2.0,4.0,639.0,1.0,0.0,Bipto,Wrenchmuller,NaN,NaN,6
20785,26044505,/m/0b6m67n,The Flying Serpent,1946,NaN,59.0,{},['United States of America'],"['Thriller', 'B-movie', 'Horror']",NaN,...,0.0,3.0,37.0,1.0,0.0,Andrew,Forbes,NaN,NaN,1
20786,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","['France', 'United States of America']","['Parody', 'Americana', 'Comedy']",85%,...,2.0,1.0,252.0,1.0,1.0,Carla,Walt,F,Julia Stiles,7
20787,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},['United States of America'],"['Thriller', 'Erotic thriller', 'Psychological...",38%,...,1.0,9.0,611.0,6.0,8.0,Greenhill,Haines,M,Don Johnson,6


## Data from summaries

### All movies

In [6]:
df_summaries = pd.read_csv('./Data/plot_summaries.txt', sep='\t', header=None,
                 names=["Wikipedia_movie_ID", "Summary"])
df_summaries['Summary'] = df_summaries['Summary'].str.replace('"', "'").replace('\\r', '')
df_summaries

,Wikipedia_movie_ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


### Only US movies

In [7]:
#create list
values = df_movie_usa_summary['Wikipedia_movie_ID']

# Use the 'isin()' function to create a Boolean Series
mask = df_summaries['Wikipedia_movie_ID'].isin(values)

# Use the Boolean Series to filter the dataframe
df_summaries_usa = df_summaries[mask].sort_values(by=['Wikipedia_movie_ID'])

In [8]:
df_summaries_usa

,Wikipedia_movie_ID,Summary
6038,3217,"After being pulled through a time portal, Ash ..."
20555,3333,The film follows two juxtaposed families: the...
39373,3746,"{{Hatnote}} In Los Angeles, November 2019, ret..."
13609,3837,"In the American Old West of 1874, construction..."
18046,3947,Jeffrey Beaumont returns to his logging home ...
...,...,...
10500,37067980,A defecting Russian diplomat has arranged to m...
19602,37210334,The movie is based upon a play by the same nam...
8984,37257312,Billy Jenks leaves his small town to find suc...
20723,37373877,"According to Horrorfest Online, six people wit..."


In [9]:
df_summaries_usa['Wikipedia_movie_ID'][10000]

4996220

# NLP analysis

## Iterative process

In [10]:
## DEFINE NLP PIPELINE
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('coreferee')

## INITIALIZE RESULTS DATAFRAME
df_character_usa_summary_results = pd.DataFrame()
df_movie_usa_summary_results = pd.DataFrame()
error = 0
error_list = []

## DEFINE THE RANGE
range_min = 0
range_max = 1 #len(df_summaries_usa['Wikipedia_movie_ID'])

## INITIALIZE EXPORT DATAFRAME
folder = '/NLP/'
folder_nlp = '/raw/'
x = 0
y = 0
for csv_file in os.listdir(os.path.abspath("") + folder):
    path_1 = 'df_character_usa_summary_results_[' + str(range_min) + '_' + str(range_max) + ']_'
    path_2 = 'df_movie_usa_summary_results_[' + str(range_min) + '_' + str(range_max) + ']_'
    x = max(int(csv_file[-5:-4]) + 1, x + 1) if path_1 in str(csv_file) else x
    y = max(int(csv_file[-5:-4]) + 1, y + 1) if path_2 in str(csv_file) else y

def get_spacy_doc_list():
    for file in os.listdir('.' + folder + folder_nlp):
        filename = os.fsdecode(file)
        yield spacy.tokens.Doc(spacy.vocab.Vocab()).from_disk('.' + folder + folder_nlp + filename)

## LOOP OVER THE MOVIES
for wik_id in tqdm(df_summaries_usa['Wikipedia_movie_ID'].iloc[range_min:range_max]):
    try:
        ## GET THE INPUTS FROM THE DATAFRAMES
        #Summary
        summary = " ".join(df_summaries_usa[df_summaries_usa['Wikipedia_movie_ID'] == wik_id]['Summary'].values[0].split())
        #List of characters
        characters = pd.DataFrame(df_character_usa_summary[df_character_usa_summary['Wikipedia_movie_ID'] == wik_id]).reset_index(drop=True)
        #Gender of characters
        gender = pd.DataFrame(df_character_usa_summary[df_character_usa_summary['Wikipedia_movie_ID'] == wik_id]['Actor_gender'])

        #Perform NLP analysis
        doc = nlp(summary, disable=[])
        chains = doc._.coref_chains

        ## GET ALL THE COREFERENCES
        #Get the main corefs
        list = []
        for chain in chains:
            for item in chain:
                for i in str(item).split(","):
                    list += [str(i).replace("[","").replace("]","").replace(" ","")]
        list_main = [ref for ref in list if doc._.coref_chains.resolve(doc[int(ref)]) == None]
        df_list_main = pd.DataFrame({"Main_coref": list_main, "Main_actors": [str(doc[int(item)]) for item in list_main]})
        df_list_main.drop_duplicates(subset = ["Main_actors"], keep ='first', inplace = True)
        list_main = df_list_main['Main_coref']

        #Get all the other corefs
        list_tokens = [ref for ref in list if doc._.coref_chains.resolve(doc[int(ref)]) != None]

        #Create the dataframe with all of them with a single row per main coref
        list_res = []
        for item in list_main:
            list_tmp = [str(doc[int(item)])]
            for item_2 in list_tokens:
                corefs = doc._.coref_chains.resolve(doc[int(item_2)])
                for coref in corefs:
                    if str(coref) == str(doc[int(item)]):
                        list_tmp += [str(doc[int(item_2)])]
            list_res += [list_tmp]
        df_corefs = pd.DataFrame(list_res).rename(columns={0: "Main_coref"})

        ## GET THE REAL CHARACTERS 
        df_simi = df_corefs

        def simil(word, choices, df_id):
            if utils.full_process(str(word)):
                # wont execute and not produce a warning
                simi = [[0,0]] if len(choices) == 0 else process.extract(str(word), choices, limit=1)
                return (simi[0][0], df_id.loc[simi[0][2]]) if simi[0][1] > 80 else (word, None)
            else:
                return (word, None)

        df_simi_1 = pd.DataFrame(df_simi["Main_coref"].apply(lambda x: simil(x,characters['Character_name'],characters['Freebase_character_ID'])[0]))
        df_simi_2 = pd.DataFrame(df_simi["Main_coref"].apply(lambda x: simil(x,characters['Character_name'],characters['Freebase_character_ID'])[1]))
        df_simi_3 = pd.DataFrame(df_simi["Main_coref"].apply(lambda x: simil(x,characters['Character_name'],characters['Actor_gender'])[1]))
        df_simi.insert(1, "Character_name", df_simi_1)
        df_simi.insert(2, "Freebase_character_ID", df_simi_2)
        df_simi.insert(3, "Actor_gender", df_simi_3)
        df_simi.insert(4, "Wikipedia_movie_ID", wik_id)

        ## GET THE GENDER
        df_gender = df_simi
        male_pronoums = ["he", "him", "his", "himself"]
        female_pronoums = ["she", "her", "hers", "herself"]
        list = []
        index = df_gender.columns.get_loc(1)

        for i in np.arange(0, len(df_gender)):
            male_test = df_gender[df_gender.columns[index:]].loc[i].astype(str).str.lower().isin(male_pronoums).sum()
            female_test = df_gender[df_gender.columns[index:]].loc[i].astype(str).str.lower().isin(female_pronoums).sum()
            list += ["M"] if (male_test > female_test) and (male_test > 0) else ["F"] if (female_test > male_test) and (female_test > 0) else [None]

        df_gender_1 = pd.DataFrame(list, columns=['Actor_gender'])
        df_gender['Actor_gender'] = df_gender['Actor_gender'].fillna(df_gender_1['Actor_gender'])

        ## GET THE MAIN ACTORS
        #define the dataframes with the main actors
        df_actor = df_gender
        df_actor = df_actor.dropna(subset=['Actor_gender'])
        df_actor_main = df_actor[df_actor.columns[0:index]]
        df_actor_count = df_actor[df_actor.columns[index:]]
        df_actor_count = df_actor_count.join(pd.DataFrame({"Coref_count": df_actor_count.notna().sum(axis=1)}))
        df_actor = df_actor_main.join(df_actor_count['Coref_count']).groupby('Character_name').aggregate({'Freebase_character_ID': 'first', 'Actor_gender': 'first', 'Wikipedia_movie_ID': 'first', 'Coref_count': 'sum'}).reset_index()

        #sort by number of coreferences
        df_actor = df_actor.sort_values('Coref_count', ascending=False)
        list_actor = np.arange(0,len(df_actor))
        df_actor['Main_actor'] = list_actor
        df_actor

        ## GET THE NEW SUMMARY
        list_new_words = []
        for word in doc:
            if chains.resolve(word)==None:
                if (df_simi['Main_coref'].astype(str) == word.text).any():
                    list_new_words += [str(df_simi[df_simi['Main_coref'].astype(str) == word.text]['Character_name'].iloc[0])]
                else:
                    list_new_words += [word.text]
            else:
                str_tmp = ""
                for res in chains.resolve(word):   
                    str_tmp += " " + str(df_simi[df_simi['Main_coref'].astype(str) == res.text]['Character_name'].iloc[0]) + " "
                str_tmp = str_tmp.replace("  ", " and ")[1:-1]
                list_new_words += [str_tmp]
        summary_coref = Doc(doc.vocab, words=list_new_words, spaces=None)

        ## GET THE ATTRIBUTES USING PARSING AND DEPENDENCIES
        #Initialize the dataframe to store the results
        df_character_usa_summary_tmp = df_actor.drop_duplicates(subset=['Character_name', 'Wikipedia_movie_ID']).reset_index(drop=True)
        #write to summaries
        df_movie_usa_summary_tmp = pd.DataFrame({'Wikipedia_movie_ID': wik_id, 'Summary': summary, 'Summary_coref': str(summary_coref)}, index=[0])
        df_movie_usa_summary_results = pd.concat([df_movie_usa_summary_results, df_movie_usa_summary_tmp]).reset_index(drop=True)
        df_movie_usa_summary_results.reset_index(drop=True)

        # parse the text using spaCy
        doc = nlp(summary_coref, disable=['coreferee'])

        #displacy.render(doc, style='dep', jupyter=True)

        # iterate through the tokens and get the adjectives related to the target token
        target_tokens = df_character_usa_summary_tmp['Character_name'].astype(str)
        list_adjectives = []
        list_verbs = []
        list_attributes = []
        for target_token in target_tokens:
            adjectives = []
            verbs = []
            attributes = []
            for token in doc:
                if token.text == target_token:
                    for left in token.lefts:
                        if left.pos_ == "ADJ":
                            adjectives.append(left.lemma_)
                    for right in token.rights:
                        if right.pos_ == "ADJ":
                            adjectives.append(right.lemma_)
                    for child in token.children:
                        if child.pos_ == "VERB":
                            verbs.append(child.lemma_)
                        elif child.pos_ == "NOUN":
                            attributes.append(child.lemma_)
                    i = 0
                    for parent in token.ancestors:
                        if parent.pos_ == "VERB" and token.dep_ == "nsubj" and i == 0:
                            verbs.append(parent.lemma_)
                        elif parent.pos_ == "AUX" and token.dep_ == "nsubj" and i == 0:
                            for granparent in parent.children:
                                if granparent.dep_ == "attr":
                                    attributes.append(granparent.lemma_)
                                    for granparent_left in granparent.lefts:
                                        if granparent_left.pos_ == "ADJ":
                                            adjectives.append(granparent_left.lemma_)
                                    for granparent_right in granparent.rights:
                                        if granparent_right.pos_ == "ADJ":
                                            adjectives.append(granparent_right.lemma_)
                        i += 1
            list_adjectives.append(adjectives)
            list_verbs.append(verbs)
            list_attributes.append(attributes)
        df_character_usa_summary_tmp['Adjectives'] = list_adjectives
        df_character_usa_summary_tmp['Verbs'] = list_verbs
        df_character_usa_summary_tmp['Attributes'] = list_attributes
        df_character_usa_summary_results = pd.concat([df_character_usa_summary_results, df_character_usa_summary_tmp]).reset_index(drop=True)

        # WRITE DATAFRAMES TO CSV FILES
        path_character = '.' + folder + 'df_character_usa_summary_results_[' + str(range_min) + '_' + str(range_max) + ']_' + str(x) + '.csv.zip'
        path_movie = '.' + folder + 'df_movie_usa_summary_results_[' + str(range_min) + '_' + str(range_max) + ']_' + str(y) + '.csv.zip'
        df_character_usa_summary_results.to_csv(path_character, compression={'method': 'zip'}, index=False, sep = ';')
        df_movie_usa_summary_results.to_csv(path_movie, compression={'method': 'zip'}, index=False, sep = ';')

        # WRITE NLP RESULTS TO BIN FILES
        doc.to_disk('.' + folder + folder_nlp + 'nlp_raw_' + str(wik_id) + '.bin')

    except:
        error += 1
        error_list += [wik_id]
        pd.DataFrame({"Error": error_list}).to_csv('.' + folder + 'errors_[' + str(range_min) + '_' + str(range_max) + ']_' + str(x) + '.csv', index=False, sep = ';')
    else:
        error += 0
print("Number of errors: " + str(error))

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]

Number of errors: 0


## Results verification

In [11]:
# ## READ THE NLP RESULTS
# for doc in get_spacy_doc_list():
#     for token in doc:
#         test = token.text #do what you want

In [12]:
df_character_usa_summary_results

,Character_name,Freebase_character_ID,Actor_gender,Wikipedia_movie_ID,Coref_count,Main_actor,Adjectives,Verbs,Attributes
0,Ash Williams,/m/06_krl,M,3217,44,0,[],"[land, fight, grow, return, start, enter, duck...","[torment, king]"
1,Duke Henry the Red,/m/0h5tv3s,M,3217,3,1,[],[capture],[]
2,Sheila,/m/0h36j9w,F,3217,2,2,[],[],[sister]
3,Lord Arthur,/m/0h5tv35,M,3217,1,3,[],[],[]


In [13]:
df_movie_usa_summary_results

,Wikipedia_movie_ID,Summary,Summary_coref
0,3217,"After being pulled through a time portal, Ash ...","After being pulled through a time portal , Ash..."


In [14]:
df_character_usa_summary_results[df_character_usa_summary_results['Actor_gender'] == 'F']

,Character_name,Freebase_character_ID,Actor_gender,Wikipedia_movie_ID,Coref_count,Main_actor,Adjectives,Verbs,Attributes
2,Sheila,/m/0h36j9w,F,3217,2,2,[],[],[sister]


In [15]:
displacy.render(doc, style='dep', jupyter=True)

## Results

### Import the characters results

In [16]:
df1 = pd.read_csv('./NLP/df_character_usa_summary_results_[0_1000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df2 = pd.read_csv('./NLP/df_character_usa_summary_results_[1000_2000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df3 = pd.read_csv('./NLP/df_character_usa_summary_results_[2000_10000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df4 = pd.read_csv('./NLP/df_character_usa_summary_results_[10000_20788]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df_character_nlp = pd.concat([df1, df2, df3, df4], axis = 0)

#write to csv file
df_character_nlp.to_csv('./NLP/df_character_nlp.csv.zip', compression={'method': 'zip'}, index = False, sep = ';')

print("FOR ALL THE CHARACTERS")

#get a list of all the adjectives
list_adjectives = df_character_nlp['Adjectives'][df_character_nlp['Adjectives'] != "[]"].reset_index(drop = True)
list_adjectives = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_adjectives]
list_adjectives = [item for sublist in list_adjectives for item in sublist]
print("The number of adjectives is : " + str(len(list_adjectives)))

#get a list of all the verbs
list_verbs = df_character_nlp['Verbs'][df_character_nlp['Verbs'] != "[]"].reset_index(drop = True)
list_verbs = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_verbs]
list_verbs = [item for sublist in list_verbs for item in sublist]
print("The number of verbs is : " + str(len(list_verbs)))

#get a list of all the attributes
list_attributes = df_character_nlp['Attributes'][df_character_nlp['Attributes'] != "[]"].reset_index(drop = True)
list_attributes = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_attributes]
list_attributes = [item for sublist in list_attributes for item in sublist]
print("The number of attributes is : " + str(len(list_attributes)))

print("-"*100)

print("FOR THE MALE CHARACTERS")

df_character_nlp_male = df_character_nlp[df_character_nlp['Actor_gender'] == "M"]

#get a list of all the adjectives
list_adjectives_M = df_character_nlp_male['Adjectives'][df_character_nlp_male['Adjectives'] != "[]"].reset_index(drop = True)
list_adjectives_M = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_adjectives_M]
list_adjectives_M = [item for sublist in list_adjectives_M for item in sublist]
print("The number of adjectives is : " + str(len(list_adjectives_M)))

#get a list of all the verbs
list_verbs_M = df_character_nlp_male['Verbs'][df_character_nlp_male['Verbs'] != "[]"].reset_index(drop = True)
list_verbs_M = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_verbs_M]
list_verbs_M = [item for sublist in list_verbs_M for item in sublist]
print("The number of verbs is : " + str(len(list_verbs_M)))

#get a list of all the attributes
list_attributes_M = df_character_nlp_male['Attributes'][df_character_nlp_male['Attributes'] != "[]"].reset_index(drop = True)
list_attributes_M = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_attributes_M]
list_attributes_M = [item for sublist in list_attributes_M for item in sublist]
print("The number of attributes is : " + str(len(list_attributes_M)))

print("-"*100)

print("FOR THE FEMALE CHARACTERS")

df_character_nlp_female = df_character_nlp[df_character_nlp['Actor_gender'] == "F"]

#get a list of all the adjectives
list_adjectives_F = df_character_nlp_female['Adjectives'][df_character_nlp_female['Adjectives'] != "[]"].reset_index(drop = True)
list_adjectives_F = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_adjectives_F]
list_adjectives_F = [item for sublist in list_adjectives_F for item in sublist]
print("The number of adjectives is : " + str(len(list_adjectives_F)))

#get a list of all the verbs
list_verbs_F = df_character_nlp_female['Verbs'][df_character_nlp_female['Verbs'] != "[]"].reset_index(drop = True)
list_verbs_F = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_verbs_F]
list_verbs_F = [item for sublist in list_verbs_F for item in sublist]
print("The number of verbs is : " + str(len(list_verbs_F)))

#get a list of all the attributes
list_attributes_F = df_character_nlp_female['Attributes'][df_character_nlp_female['Attributes'] != "[]"].reset_index(drop = True)
list_attributes_F = [item.replace('[', '').replace(']', '').replace("'", '').split(', ') for item in list_attributes_F]
list_attributes_F = [item for sublist in list_attributes_F for item in sublist]
print("The number of attributes is : " + str(len(list_attributes_F)))

FOR ALL THE CHARACTERS
The number of adjectives is : 29744
The number of verbs is : 332304
The number of attributes is : 65644
----------------------------------------------------------------------------------------------------
FOR THE MALE CHARACTERS
The number of adjectives is : 19383
The number of verbs is : 230527
The number of attributes is : 44397
----------------------------------------------------------------------------------------------------
FOR THE FEMALE CHARACTERS
The number of adjectives is : 10361
The number of verbs is : 101777
The number of attributes is : 21247


### All characters

In [17]:
df_character_nlp

,Character_name,Freebase_character_ID,Actor_gender,Wikipedia_movie_ID,Coref_count,Main_actor,Adjectives,Verbs,Attributes
0,Ash Williams,/m/06_krl,M,3217,44,0,[],"['land', 'fight', 'grow', 'return', 'start', '...","['torment', 'king']"
1,Duke Henry the Red,/m/0h5tv3s,M,3217,3,1,[],['capture'],[]
2,Sheila,/m/0h36j9w,F,3217,2,2,[],[],['sister']
3,Lord Arthur,/m/0h5tv35,M,3217,1,3,[],[],[]
4,Cameron,NaN,M,3333,15,0,[],"['idolize', 'gain', 'meet', 'persuade', 'go', ...",['father']
...,...,...,...,...,...,...,...,...,...
50450,Johnna,NaN,M,37210334,2,2,[],['commit'],[]
50451,poet,NaN,M,37210334,1,3,[],[],[]
50452,Billy,NaN,M,37257312,8,0,[],"['try', 'get', 'get']",[]
50453,Jenks,NaN,M,37257312,3,1,[],"['leave', 'settle', 'get']",[]


### Male characters

In [18]:
df_character_nlp_male

,Character_name,Freebase_character_ID,Actor_gender,Wikipedia_movie_ID,Coref_count,Main_actor,Adjectives,Verbs,Attributes
0,Ash Williams,/m/06_krl,M,3217,44,0,[],"['land', 'fight', 'grow', 'return', 'start', '...","['torment', 'king']"
1,Duke Henry the Red,/m/0h5tv3s,M,3217,3,1,[],['capture'],[]
3,Lord Arthur,/m/0h5tv35,M,3217,1,3,[],[],[]
4,Cameron,NaN,M,3333,15,0,[],"['idolize', 'gain', 'meet', 'persuade', 'go', ...",['father']
6,Ben,NaN,M,3333,11,2,[],"['fight', 'find', 'rescue']","['servant', 'other']"
...,...,...,...,...,...,...,...,...,...
50448,Karen,NaN,M,37210334,4,0,[],['prove'],[]
50450,Johnna,NaN,M,37210334,2,2,[],['commit'],[]
50451,poet,NaN,M,37210334,1,3,[],[],[]
50452,Billy,NaN,M,37257312,8,0,[],"['try', 'get', 'get']",[]


### Female characters

In [19]:
df_character_nlp_female

,Character_name,Freebase_character_ID,Actor_gender,Wikipedia_movie_ID,Coref_count,Main_actor,Adjectives,Verbs,Attributes
2,Sheila,/m/0h36j9w,F,3217,2,2,[],[],['sister']
5,Elsie,NaN,F,3333,12,1,[],"['take', 'break', 'learn', 'go', 'break', 'fal...",['son']
8,Flora,NaN,F,3333,7,4,[],"['warn', 'jump', 'leap', 'lie']",[]
14,Rachael,/m/02nthch,F,3746,16,2,"['assistant', 'experimental']","['visit', 'leave', 'disappear', 'kill', 'try',...","['replicant', 'replicant']"
17,Pris,/m/02nthb7,F,3746,3,5,[],"['meet', 'live']",[]
...,...,...,...,...,...,...,...,...,...
50434,girlfriend,NaN,F,36814246,4,3,[],['tell'],[]
50439,infant,NaN,F,36814246,2,8,[],"['refuse', 'leave']",[]
50446,William’s Mother,/m/03bqs07,F,36956792,1,6,[],"['allow', 'call']","['old angus', 'son']"
50449,Ivy,NaN,F,37210334,3,1,[],"['live', 'plan']",[]


### Import the summaries results

In [20]:
df1 = pd.read_csv('./NLP/df_movie_usa_summary_results_[0_1000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df2 = pd.read_csv('./NLP/df_movie_usa_summary_results_[1000_2000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df3 = pd.read_csv('./NLP/df_movie_usa_summary_results_[2000_10000]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df4 = pd.read_csv('./NLP/df_movie_usa_summary_results_[10000_20788]_0.csv.zip', sep = ';', encoding = 'utf-8').reset_index(drop = True)
df_movie_usa_summary_nlp = pd.concat([df1, df2, df3, df4], axis = 0)

#write to csv file
df_movie_usa_summary_nlp.to_csv('./NLP/df_movie_usa_summary_nlp.csv.zip', compression={'method': 'zip'}, index = False, sep = ';')

df_movie_usa_summary_nlp

,Wikipedia_movie_ID,Summary,Summary_coref
0,3217,"After being pulled through a time portal, Ash ...","After being pulled through a time portal , Ash..."
1,3333,The film follows two juxtaposed families: the ...,The film follows two juxtaposed families : the...
2,3746,"{{Hatnote}} In Los Angeles, November 2019, ret...","{ { Hatnote } } In Los Angeles , November 2019..."
3,3837,"In the American Old West of 1874, construction...","In the American Old West of 1874 , constructio..."
4,3947,Jeffrey Beaumont returns to his logging home t...,Jeffrey Beaumont Jeffrey Beaumont returns to J...
...,...,...,...
12898,37067980,A defecting Russian diplomat has arranged to m...,A defecting Russian diplomat has arranged to m...
12899,37210334,The movie is based upon a play by the same nam...,The movie is based upon a play by the same nam...
12900,37257312,Billy Jenks leaves his small town to find succ...,Billy Jenks leaves Jenks small town to find su...
12901,37373877,"According to Horrorfest Online, six people wit...","According to Horrorfest Online , six people wi..."
